# Drille ned i lavere aggregater i linje graf

Dette er ett eksempel på hvordan man kan benytte Plotly og Dash til å lage 
interaktiv graf til å kunne drille nedover i aggregatene sine.
Eksempelet tar for seg data fra Utenrikshandel med varer som hentes rett fra Statistikkbank APIet.
Enhver kan derfor kjøre dette notebooket for å teste det ut.

## Importere diverse pakker

In [ ]:
import plotly.express as px
import datetime
import dash
from jupyter_dash import JupyterDash
from dash import html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate
import json

## Importere interne og prosjektinterne pakker

In [ ]:
from statbankpy import apidata as sb
from config_ssbapi import uhv_payload_sitc3, uhv_payload_sitc1, uhv_payload_sitc2

In [ ]:
help(sb)

## Hente data fra ssb.no apiet, samt justere dataene noe

In [ ]:
uhvdf = sb('08818', uhv_payload_sitc3, True)
uhvdf = uhvdf.rename(str.lower, axis='columns')
uhvdf = uhvdf.rename(columns={
    "sitc": "sitc3",
    "value": "verdi",
    "måned": "mnd",
    "impeks": "imex",
    "import/eksport": "impeks"})
uhvdf["sitc2"] = uhvdf.sitc3.str[:2]
uhvdf["sitc1"] = uhvdf.sitc3.str[:1]
uhvdf["year"] = uhvdf.mnd.str[:4]
fromyear = datetime.date.today().year - 10
uhvdf = uhvdf[uhvdf.year >= f"{fromyear}"]
#uhvdf

Henter ut tekster til de andre aggregatene. Kunne ha benyttet klass her...

In [ ]:
sitc1 = sb('08806', uhv_payload_sitc1, True)
sitc1 = sitc1.rename(str.lower, axis='columns')
sitc1 = sitc1.rename(columns={"sitc": "sitc1"})
sitc1 = sitc1[['sitc1', 'varegruppe']].drop_duplicates().reset_index(drop=True)
sitc1 = dict(zip(sitc1.sitc1.to_list(), sitc1.varegruppe.to_list()))
#sitc1

In [ ]:
sitc2 = sb('08806', uhv_payload_sitc2, True)
sitc2 = sitc2.rename(str.lower, axis='columns')
sitc2 = sitc2.rename(columns={"sitc": "sitc1"})
sitc2 = sitc2[['sitc1', 'varegruppe']].drop_duplicates().reset_index(drop=True)
sitc2 = dict(zip(sitc2.sitc1.to_list(), sitc2.varegruppe.to_list()))
#sitc2

In [ ]:
sitc3 = uhvdf[['sitc3', 'varegruppe']].drop_duplicates().reset_index(drop=True)
sitc3 = dict(zip(sitc3.sitc3.to_list(), sitc3.varegruppe.to_list()))
#sitc3

Setter sammen alt til en og samme dataframe

In [ ]:
uhvdf['sitc1txt'] = uhvdf['sitc1'].map(sitc1)
uhvdf['sitc2txt'] = uhvdf['sitc2'].map(sitc2)
#uhvdf

## Lager første figur

In [ ]:
df = uhvdf.groupby(['impeks', 'mnd']).sum().reset_index()
fig = px.line(df, x="mnd", y="verdi", color='impeks')
fig

## Dashboard med mulighet for å drille i graf

In [ ]:
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

OBS! VENT 2-3 sekunder her før du kjører neste celle!

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
def get_aggregater():
    aggregater = ['root', 'impeks', 'sitc1', 'sitc2', 'sitc3']
    return aggregater

In [ ]:
layout_style = {
    'backgroundColor': '#E3F1E6',
    'height': '100vh'
}

In [ ]:
app = JupyterDash(assets_folder='assets', external_stylesheets=[dbc.themes.BOOTSTRAP])
app.config.suppress_callback_exceptions = True
porten = 8777

app.layout = html.Div([
    html.H1('Norwegian external trade last 10 years', style={'text-align': 'center'})
    ,
    html.Hr()
    ,
    dcc.Markdown(id='where-am-i', children="""Hvor i aggregatet: """)
    ,
    dcc.Markdown(id='varetxt', children="Vareforklaring: ")
    ,
    html.Hr()
    ,
    dbc.Button('🡠', id='back-button', size="lg",
               className='me-1', style={'display': 'none'}, n_clicks=0)
    ,
    dcc.Graph(id='the-graph', figure=fig)
    ,
    html.H5('ClickData')
    ,
    html.Div(id='click-data')
    ,
    html.H5('Click Triggered')
    ,
    html.Div(id='click-trigg')
    ,
    dcc.Store(id='drillagg', data=['root'], storage_type='memory')
    ,
    dcc.Store(id='my-agg-click', data={}, storage_type='memory')
], style=layout_style)


@app.callback(
    [Output('the-graph', 'figure'),
     Output('back-button', 'style'),
     Output('back-button', 'n_clicks'),
     Output('where-am-i', 'children'),
     Output('drillagg', 'data'),
     Output('my-agg-click', 'data'),
     Output('varetxt', 'children')],
    [Input('the-graph', 'clickData'),
     Input('back-button', 'n_clicks')],
    [State('drillagg', 'data'),
     State('my-agg-click', 'data')],
    prevent_initial_call=True)
def drilling_in_graph(clickdata, bclick, colagg, drillagg):
    # Henter ut data fra det vi har trykket på i dashboardet
    trigger_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    curve_trig = clickdata['points'][0]['curveNumber']
    df = uhvdf.copy()
    # Hvis grafen blir trykket på
    if 'the-graph' in trigger_id:
        # Hvor er vi i aggregatet
        aggwhere = colagg
        hvor = aggwhere[-1]
        agglist = get_aggregater()
        drilldict = drillagg
        # hvis vi står i startposisjon
        if hvor == aggwhere[0]:
            # Ordner praktiske ting
            nextagg = agglist.index(hvor) + 1
            nextagg = agglist[nextagg]
            groupagg = agglist.index(hvor) + 2
            groupagg = agglist[groupagg]
            drill = sorted(list(df[nextagg].unique()))[curve_trig]
            aggwhere.append(nextagg)
            drilldict[nextagg] = drill
            # Order data og figur
            df = df[df[nextagg] == drill]
            dff = df.groupby([groupagg, 'mnd']).sum().reset_index()
            fig = px.line(dff, x="mnd", y="verdi", color=groupagg)
            # Oppdaterer markdown teksten
            md = f"Hvor i aggregatet: {drill}"
            vtxt = f"Vareforklaring: Vareførsel {drill}"
            return fig, {'display': 'block'}, 0, md, aggwhere, drilldict, vtxt
        # hvis vi uansett ikke kan drille videre
        elif hvor == agglist[-2]:
            # Ettersom vi ikke har mer data på et lavere aggregat å vise fram
            raise PreventUpdate
        else:
            # Ordner data og praktiske ting
            for i in drilldict:
                df = df[df[i] == drilldict.get(i)]
            nextagg = agglist.index(hvor) + 1
            nextagg = agglist[nextagg]
            groupagg = agglist.index(hvor) + 2
            groupagg = agglist[groupagg]
            drill = sorted(list(df[nextagg].unique()))[curve_trig]
            df = df[df[nextagg] == drill]
            aggwhere.append(nextagg)
            drilldict[nextagg] = drill
            # ferdigstiller figur
            dff = df.groupby([groupagg, 'mnd']).sum().reset_index()
            fig = px.line(dff, x="mnd", y="verdi", color=groupagg)
            # Oppdaterer markdown teksten
            mdtxt = [i[1] for i in drilldict.items()]
            mdtxt = ' -> '.join(mdtxt)
            md = f"Hvor i aggregatet: {mdtxt}"
            if nextagg == 'impeks':
                vtxt = df.impeks.unique()[0]
                vtxt = f"Vareforklaring: Vareførsel {vtxt}"
            else:
                vtxt = df[f"{nextagg}txt"].unique()[0]
                vtxt = f"Vareforklaring: varetekst {vtxt}"
            return fig, {'display': 'block'}, 0, md, aggwhere, drilldict, vtxt
    # Hvis tilbake-knappen blir trykket på
    elif bclick == 1:
        aggwhere = colagg
        hvor = aggwhere[-1]
        agglist = get_aggregater()
        drilldict = drillagg
        backto = agglist.index(hvor) - 1
        backto = agglist[backto]
        del aggwhere[-1]
        drilldict.popitem()
        if backto == 'root':
            df = df.groupby(['impeks', 'mnd']).sum().reset_index()
            fig = px.line(df, x="mnd", y="verdi", color='impeks')
            md = "Hvor i aggregatet: "
            vtxt = "Vareforklaring: "
            return fig, {'display': 'none'}, 0, md, aggwhere, drilldict, vtxt
        else:
            for i in drilldict:
                val = drilldict.get(i)
                df = df[df[i] == val]
            whereto = drilldict.get(backto)
            df = df[df[backto] == whereto]
            dff = df.groupby([hvor, 'mnd']).sum().reset_index()
            fig = px.line(dff, x="mnd", y="verdi", color=hvor)
            mdtxt = [i[1] for i in drilldict.items()]
            mdtxt = ' -> '.join(mdtxt)
            md = f"Hvor i aggregatet: {mdtxt}"
            if backto == 'impeks':
                vtxt = df.impeks.unique()[0]
                vtxt = f"Vareforklaring: Vareførsel {vtxt}"
            else:
                vtxt = df[f"{backto}txt"].unique()[0]
                vtxt = f"Vareforklaring: varetekst {vtxt}"
            return fig, {'display': 'block'}, 0, md, aggwhere, drilldict, vtxt
    # Ellers, så skal ingenting skje
    else:
        raise PreventUpdate


@app.callback(
    [Output('click-data', 'children'),
     Output('click-trigg', 'children')],
    [Input('the-graph', 'clickData')],
    prevent_initial_call=True)
def display_click_data(clickData):
    trigger_id = dash.callback_context.triggered
    return json.dumps(clickData, indent=2), json.dumps(trigger_id)


if __name__ == '__main__':
    #app.run_server(debug=True, port=porten, mode="jupyterlab")
    app.run_server(debug=True, port=porten)

In [ ]:
# Viktig å avslutte porten enten ved å kjøre dette eller å lukke kernelen når du er ferdig:
app._terminate_server_for_port("localhost", porten)